In [ ]:
import nbdev.showdoc

In [ ]:
#default_exp export_sbt
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [ ]:
#export
from nbdev.imports import *
from fastcore.script import *
from fastcore.foundation import *
from keyword import iskeyword
import nbformat

from chisel_nbdev.export_scala import *

# Export to Scala SBT project 

> The functions that allow one to generate an SBT project from the generated .sc Scala scripts.

Each 

# Manage imports
> Ammonite style imports are not valid in SBT. It is required to convert these imports and support Scala packages.

Creates all of the folders required for SBT in the folder specified by `lib_path` in `settings.ini`.
- src
    - main
        - java
        - resource
        - scala
    - test
        - java
        - resource
        - scala

In [ ]:
#export
def init_sbt_layout():
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")
    for d in ['main', 'test']:
        for dd in ['resource', 'scala', 'java']:
            src = path/'src'/d/dd
            src.mkdir(parents=True, exist_ok=True)

In [ ]:
init_sbt_layout()

Need to convert from
```$file.^.source.load_ivy```

to
```import package.module_name```

iff `module_name` is not part of the same `package` as this file

In [ ]:
#export
_re_ammonite_import = re.compile(r'^\s*import\s+\${1}.+', re.MULTILINE | re.VERBOSE)

In [ ]:
#export 
def split_amm_imports(code):
    imports = _re_ammonite_import.findall(code)
    return imports

In [ ]:
#export
def get_import_mod_names(imports):
    mod_names = []
    for i in imports:
        try:
            l1, l2 = i.split(",")
            mod_name = l2.split(".")[0].strip()
        except ValueError:
            mod_name = i.split(".")[-1].strip()
        mod_names.append(mod_name)
        print(f"import statement: {i}, mod_name: {mod_name}")
        
    return mod_names

In [ ]:
#export
def mod_in_same_package(mod_name, package):
    mod = get_nbdev_module()
    return f'{mod_name}.sc' in mod.packages.get(package)

In [ ]:
test_eq(mod_in_same_package("ModB", "ComposedExample"), True)

In [ ]:
# cell = nbformat.v4.new_code_cell(f' import $file.^.source.load_ivy  \n some code')
cell = nbformat.v4.new_code_cell(f' import $file.^.source.load_ivy, load_ivy._  \n some code')
code = f' import $file.^.source.load_ivy, load_ivy._  \n some code \n import $file.^.lib_name.mod_name\n '
imps = split_amm_imports(code)
print(imps)
mod_names = get_import_mod_names(imps)
    
# print(check_re(cell, _re_ammonite_import).groups(0))

[' import $file.^.source.load_ivy, load_ivy._  ', ' import $file.^.lib_name.mod_name']
import statement:  import $file.^.source.load_ivy, load_ivy._  , mod_name: load_ivy
import statement:  import $file.^.lib_name.mod_name, mod_name: mod_name


In [ ]:
#export
def conv_imports(code, package):
    imports = split_amm_imports(code)
    mod_names = get_import_mod_names(imports)
    for m in mod_names:
        skip_imp = mod_in_same_package(m, package)
        print(f'{m} is in package {package} == {skip_imp}')

In [ ]:
#export
def get_mod_path(mod_name):
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")
    return path/mod_name

In [ ]:
#export
def get_mods_package(mod_name): 
    "Returns the package that this mod (.sc) is listed under"
    packages = get_nbdev_module().packages
    for p, ms in packages.items():
        for m in ms:
            if m == mod_name + '.sc': return p
    return None


In [ ]:
test_eq(get_mods_package('ModB'), 'ComposedExample')

In [ ]:
# test_nb = read_nb('import_composed_mod.ipynb')
# cell = test_nb['cells'][3]
with open(path/'ModB.sc') as f: 
    code = f.read()
    print(code)
    conv_imports(code, get_mods_package('ModB'))

// AUTOGENERATED! DO NOT EDIT! File to edit: nbs/import_chisel_mod.ipynb (unless otherwise specified).

// Cell
//default_exp ModB
//package ComposedExample

// Cell
import $file.^.nbdev.ModA, ModA._

// Cell
import $file.^.source.load_ivy

// Cell
import chisel3.{Input => Input}
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.tester.RawTester.test

// Cell
class ComposedModule extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(4.W))
        val b = Input(UInt(4.W))
        val c = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    val addr = Module(new Add)
    addr.io.a := io.a
    addr.io.b := io.b
    io.out := addr.io.out + io.c
}
import statement: import $file.^.nbdev.ModA, ModA._, mod_name: ModA
import statement: import $file.^.source.load_ivy, mod_name: load_ivy
ModA is in package ComposedExample == True
load_ivy is in package ComposedExample == Fal

# script2scala()
> Replaces the Ammonite imports with SBT compatible imports and wraps code in `Object`.

In [ ]:
#export
def script2scala(fname, package, file_path):
    "Edit a scala file to work with SBT."
    with open(fname, 'r') as f: code = f.read()
        
    # replace imports
    
    # bundle the non-import code in an Object
    
    # insert package
    
    with open(fname, 'w') as f:
#         f.write(f"// AUTOGENERATED! DO NOT EDIT! File to edit: {file_path} (unless otherwise specified).")
        f.write(code)

# create_packages
> Creates the .scala files from the .sc scripts and the package data contained in `_.nbdev.py`

In [ ]:
#export
def create_packages():
    "Create directory for each package and export respective scala scripts `files` under `modules`"
    mod = get_nbdev_module()
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")/'src'/'main'/'scala'
    for package, modules in mod.packages.items():
        pname = path/package
        pname.mkdir(parents=True, exist_ok=True)
        for mod in modules:
            mod_name = pname/f'{mod + "ala"}'
            file_path = cfg.path("lib_path")/mod
            shutil.copy(file_path, mod_name)
            script2scala(mod_name, package, file_path)
            print(f'copied {mod} -> {package}/{mod + "ala"}')

In [ ]:
create_packages()

copied ModC.sc -> ComposedExample/ModC.scala
copied ModA.sc -> ComposedExample/ModA.scala
copied ModB.sc -> ComposedExample/ModB.scala
copied NewScript2.sc -> Examples/NewScript2.scala
copied test.sc -> Examples/test.scala
copied NewScript.sc -> Examples/NewScript.scala


To support exporting SBT projects we need to be able to use packages.